# MD with OpenMM

This example runs molecular dynamics of a small molecule using AceFF1.1 as a ML potential. It uses [OpenMM](https://openmm.org/) and [OpenMM-ML](https://github.com/openmm/openmm-ml).

It uses our code from [aceff_examples](https://github.com/Acellera/aceff_examples) that implements torchmdnet as an OpenMM-ML MLPotential.


**Before running you will need to change the runtime type on Colab to GPU!**
You can change to a GPU instance on Colab by clicking `runtime`→`change runtime type` and selecting `T4 GPU` from the `Hardware accelerator` dropdown menu.

In [ ]:
# Execute this cell to setup the python env in the Colab environment
if 'google.colab' in str(get_ipython()):
    print('Running on colab')
    !pip install -q condacolab
    import condacolab
    condacolab.install_mambaforge()
    !rm -rf /usr/local/conda-meta/pinned # remove pins so we can use cuda 11.8
    import os
    os.environ["CONDA_OVERRIDE_CUDA"] = "11.8"
    !mamba install torchmd-net=*=cuda118*
    !mamba install openmm-torch=*=cuda118*
    !mamba install -c conda-forge openmm-ml
    !mamba install -c conda-forge openmmforcefields
else:
    print('Not running on colab.')
    print('Make sure you create and activate a new conda environment!')
    print('Please install the above packages.')

**Notes:**

- During this step on Colab the kernel will be restarted. This will produce the error message: "Your session crashed for an unknown reason. " This is normal and can be safely ignored. 
- Installing the necessary packages may take several minutes.

In [ ]:
# install the AceFF examples code
!pip install git+https://github.com/Acellera/aceff_examples.git

In [ ]:
# install huggingface_hub
!pip install huggingface_hub

In [ ]:
# log into HuggingFace

from huggingface_hub import login

# Create a HuggingFace token to access public gated repos: https://huggingface.co/settings/tokens
# And paste your token here (keep it secret!)
login(TOKEN)

Before you can download the model you must accept the license, please go to:
https://huggingface.co/Acellera/AceFF-1.1

In [ ]:
# download the model
from huggingface_hub import hf_hub_download

model_file_path = hf_hub_download(
    repo_id="Acellera/AceFF-1.1",
    filename="aceff_v1.1.ckpt"
)

print("Downloaded to:", model_file_path)

In [ ]:
# download the example ligand file
!wget https://raw.githubusercontent.com/Acellera/aceff_examples/refs/heads/main/notebooks/ejm_31_ligand.sdf

In [ ]:
import openmm.app as app
import openmm as mm
import openmm.unit as unit
from sys import stdout
from openff.toolkit import Molecule
from openff.toolkit import Topology as offTopology
from openff.units.openmm import to_openmm as offquantity_to_openmm
from openmmml import MLPotential
# this import registers the model with openmm-ml
from aceff_examples import torchmdnetpotential


# user supplied path
sdfname = "ejm_31_ligand.sdf"


# Load the ligand with OpenFF toolkit
ligand = Molecule.from_file(sdfname)

# make an OpenFF Topology of the ligand
ligand_off_topology = offTopology.from_molecules(molecules=[ligand])

# get the total molecule charge
molecule_charge = int(sum( [ atom.formal_charge.magnitude for atom in ligand_off_topology.atoms]))

# convert it to an OpenMM Topology
ligand_omm_topology = ligand_off_topology.to_openmm()

# get the positions of the ligand
ligand_positions = offquantity_to_openmm(ligand.conformers[0])

# get the indicies
# this is a list of list so molecules can be batched if needed
group_indices=[list(range(ligand_off_topology.n_atoms))]

# create the ML potential with AceFF
potential = MLPotential('TorchMD-NET', model_file=model_file_path, group_indices=group_indices, molecule_charges=[molecule_charge], max_num_neighbors=40)

# create the OpenMM system
system = potential.createSystem(ligand_omm_topology)

integrator = mm.LangevinMiddleIntegrator(300*unit.kelvin, 1.0/unit.picosecond, 1.0*unit.femtoseconds)
simulation = app.Simulation(ligand_omm_topology, system, integrator, platform = mm.Platform.getPlatformByName('CUDA'))

# set the positions
simulation.context.setPositions(ligand_positions)

print("Minimizing energy...")
simulation.minimizeEnergy(maxIterations=1000)

simulation.context.setVelocitiesToTemperature(300*unit.kelvin)

simulation.reporters.append(app.PDBReporter(f'traj.pdb', 1000))

simulation.reporters.append(app.StateDataReporter(stdout, 1000, step=True,
        potentialEnergy=True, temperature=True, speed=True))

print("Running simulation...")
simulation.step(10000)





